# Unified Brane-Cosmology — Real Data Minimal Fit
Drop real PTA spectrum + CMB/BBN prior here. Produces posteriors and a results PDF.

In [ ]:
import numpy as np, pandas as pd, json
import matplotlib.pyplot as plt
from scipy.optimize import minimize
pta = pd.read_csv('pta_spectrum_REAL.csv')
pri = pd.read_csv('cmb_bbn_priors_REAL.csv')
DNeff_mu, DNeff_sig = pri.iloc[0]['DeltaNeff_mean'], pri.iloc[0]['DeltaNeff_sigma']

def fbreak_from_lambda(lmbda, k=1e-4):
    return k*(lmbda**0.25)
def model_Omega(f, A1, A2, fb, n1=-2/3, n2=-1):
    return np.where(f<fb, A1*(f/fb)**n1, A2*(f/fb)**n2)
def negloglike(theta):
    loglmbda, A1, A2 = theta
    lmbda = np.exp(loglmbda)
    fb = fbreak_from_lambda(lmbda)
    Om = model_Omega(pta.f_Hz.values, A1, A2, fb)
    chi2 = np.sum(((pta.OmegaGW.values-Om)/pta.sigma_OmegaGW.values)**2)
    # simple Gaussian prior on DeltaNeff (placeholder link to lambda)
    DNeff = 3.046 + 0.0 # placeholder; model this mapping as needed
    chi2 += ((DNeff - DNeff_mu)/DNeff_sig)**2
    return 0.5*chi2
x0 = np.array([0.0, 1e-9, 1e-9])
res = minimize(negloglike, x0, method='Nelder-Mead')
best = {'log_lambda':float(res.x[0]), 'A1':float(res.x[1]), 'A2':float(res.x[2])}
with open('bestfit_real.json','w') as f: json.dump(best, f, indent=2)
fb = fbreak_from_lambda(np.exp(res.x[0]))
ff = np.logspace(np.log10(pta.f_Hz.min()/3), np.log10(max(pta.f_Hz.max()*3,1e-2)), 400)
Om = model_Omega(ff, res.x[1], res.x[2], fb)
plt.figure(); plt.loglog(pta.f_Hz, pta.OmegaGW, 'o'); plt.loglog(ff, Om)
plt.xlabel('f [Hz]'); plt.ylabel('Omega_GW'); plt.tight_layout(); plt.savefig('pta_fit_real.png', dpi=160)